In [146]:
import pandas as pd
from scipy.spatial.distance import cosine
from scipy import stats
import torch
from transformers import BertTokenizer, BertModel
import matplotlib.pyplot as plt

In [147]:
def find_token(mat, tok_sen):
    candidate = []
    for i, t in enumerate(tok_sen):
        if mat.startswith(t):
            idx = [i]
            can = t
            tmp = i
            if can == mat:
                candidate.extend(idx)
            if i == len(tok_sen) - 1:
                break
            while True:
                can += tok_sen[tmp + 1].replace('##', '')
                if not mat.startswith(can):
                    break
                idx.append(tmp + 1)
                if can == mat:
                    candidate.extend(idx)
                    break
                if tmp == len(tok_sen) - 2:
                    break
                else:
                    tmp += 1
    return candidate

In [148]:
# C:/Users/Lenovo/Desktop/EnergyBERT/mat_bert_cased
# bert-cased
tokenizer = BertTokenizer.from_pretrained('C:/Users/Lenovo/Desktop/EnergyBERT/mat_bert_cased', do_lower_case=False)
model = BertModel.from_pretrained('C:/Users/Lenovo/Desktop/EnergyBERT/mat_bert_cased', output_hidden_states=True).eval()

Some weights of the model checkpoint at C:/Users/Lenovo/Desktop/EnergyBERT/mat_bert_cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at C:/Users/Lenovo/Desktop/EnergyBERT/mat_bert_cased and are newly initialized

In [149]:
def get_embedding(mat, text, tokenizer, model):
    tokenized_text = tokenizer.tokenize(text)
    need_idx = find_token(mat, tokenized_text)
    if len(need_idx)==0:
        return False
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    with torch.no_grad():
        outputs = model(tokens_tensor)
        # hidden_states has 4 dimensions: 13 layer, 1 batch, x token number, 768 hidden unit
        hidden_states = outputs[2]
    token_embeddings = torch.stack(hidden_states, dim=0)
    # remove batch dimension
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    # size: x token, 13 layer, 768 hidden unit
    token_embeddings = token_embeddings.permute(1,0,2)
    token_vecs_sum = []
    for token in token_embeddings:
        # last 4 layer
        sum_vec = torch.sum(token[2:3], dim=0)
        # sum_vec: [768]
        token_vecs_sum.append(sum_vec)
    embeddings = []
    for i in need_idx:
        embeddings.append(token_vecs_sum[i])
    sentence_embedding = torch.mean(torch.stack(embeddings), 0)
    return sentence_embedding

In [150]:
import random
from tqdm import tqdm

random.seed(0)

def get_final_embedding(mat, tokenizer, model, zt_json):
    tmp_embedding = []
    for zj in zt_json.keys():
        if mat == zt_json[zj]['ori']:
            tmp_sentence = zt_json[zj]['sens']
    if len(tmp_sentence)>100:
        sentence = random.sample(tmp_sentence, 100)
    else:
        sentence = tmp_sentence
    print(len(sentence), sentence[:5])
    for ts in tqdm(sentence):
        embedding = get_embedding(mat, ts, tokenizer, model)
        if type(embedding)!=bool:
            tmp_embedding.append(embedding)
    if tmp_embedding==[]:
        return get_embedding(mat, mat, tokenizer, model)
    else:
        return torch.mean(torch.stack(tmp_embedding), 0)

In [151]:
import json

file_path = 'for_spearman/'
file_name = 'zt_has_sen.txt'
with open('zt_json.json', 'r', encoding='utf-8') as f:
    zt_json = json.load(f)

In [152]:
df = pd.read_csv(file_path+file_name, sep='\t', header=None, names=['name', 'value'])

In [153]:
# thermoelectric_embedding = get_embedding("thermoelectric", "thermoelectric", tokenizer, model)
with open('context_zt/thermo_sens.json', 'r', encoding='utf-8') as f:
    thermo_sen = json.load(f)
thermo_sen = random.sample(thermo_sen, 100)
tmp_embedding = []
for ts in tqdm(thermo_sen):
    embedding = get_embedding('thermoelectric', ts, tokenizer, model)
    if type(embedding)!=bool:
        tmp_embedding.append(embedding)
thermoelectric_embedding = torch.mean(torch.stack(tmp_embedding), 0)

100%|██████████| 100/100 [00:07<00:00, 13.01it/s]


In [154]:
record = {}
for i, n in enumerate(df['name']):
    similarity = 1 - cosine(get_final_embedding(n, tokenizer, model, zt_json), thermoelectric_embedding)
    token_num = len(tokenizer.tokenize(n))
    record[n] = {}
    record[n]['sim'] = similarity
    record[n]['token_num'] = token_num
    record[n]['thermo_rank'] = i+1

  3%|▎         | 3/100 [00:00<00:04, 23.44it/s]

100 ['By using mechanical alloying and solid-state reaction methods to control the Te doping contents in the Cu2Se1−xTex (x\xa0=\xa00.1, 0.2, 0.25, and 0.3) at a sintering temperature of 873\xa0K, we observed a structural phase transition and a change in the grain morphology at the Te content of 0.25.', 'The molar ratio of Cu/Se based on this structure is ∼1.97, suggesting the deficiency of Cu in Cu2Se.', 'However, when Cu2Se is stacked on SnSe2 either on SLG or Mo, the size of the crystallites is very low.', 'Therefore, there are only a few reports about Cu2Se thin films prepared by magnetron sputtering or the evaporation method.', 'The Bi0.01Cu2Se sample obtained the lowest thermal conductivity of 0.58\xa0Wm−1\xa0K−1, which is approximately a 33.3% reduction than that of pure Cu2Se at 873\xa0K. Figure\xa07b shows the temperature-dependent lattice thermal conductivity (κl) for BixCu2Se bulk samples.']


 12%|█▏        | 2/17 [00:00<00:01, 12.66it/s]

17 ['The polycrystalline In0.25Co4Sb12 n-type skutterudite was ball-milled to reduce its grain size and lattice thermal conductivity by enhanced phonon scattering at grain boundaries. ', 'This leads to several highly dense In0.25Co4Sb12 compacts (> 97%) with average grain sizes ( D) ranging from 400 nm to the very small value of 150 nm. ', 'ZT is nonetheless improved from ZT = 1.1 in pristine In0.25Co4Sb12 to ZT = 1.4 at 750 K in In0.25Co4Sb12 with D similar to 400 nm because in this sample, the lattice thermal conductivity is still efficiently reduced while the electron mobility remains unaffected.', 'A ZT larger than 1 was first reported in the indium partially filled skutterudite In0.25Co4Sb12 by He et\xa0al.', 'As will be further discussed, the lattice thermal conductivity is strongly affected by the grain size reduction below 400\xa0nm in In0.25Co4Sb12, leading to a large maximum ZT.']


 14%|█▍        | 2/14 [00:00<00:00, 12.27it/s]

...

 50%|█████     | 1/2 [00:00<00:00,  8.40it/s]

2 ['A number of ternary bismuth chalcogenides have been synthesized, such as KBi6.33S10 [24], K2Bi8S13 [24], β-K2Bi8Se13 [25], and K2.5Bi8.5Se14.', 'For instance, KBi6.33S10 and K2Bi8S13 [24] were synthesized using potassium sulfide and bismuth sulfide at high temperature, and the results showed that the thermal conductivity of KBi6.33S10 is significantly lower than that of Bi2Te3 and both compounds are semiconductors with an electrical conductivity of 102 S/cm at 300 K. McCarthy et al. [26] found that γ-CsBiS2 and K2Bi8Sel3 are both n-type semiconductor materials and their electrical conductivity at room temperature is 10− 2 S/cm.']


 33%|███▎      | 1/3 [00:00<00:00,  8.90it/s]

3 ["In the present work, a novel photocatalyst polyacrylamide/Ni0.02Zn0.98O (PAM/NZP) was synthesized successfully by addition of nanoparticles during polymerization of acrylamide in aqueous medium using ammonium persulfate and N,N'-methylenebis(acrylamide).", 'The effect of adsorption capacity of cross-linked polyacrylamide on photocatalytic activity of Ni0.02Zn0.98O was also studied. ', 'In this study, a selective method for removal of dyes from aqueous solutions using nanocomposites of nickel doped zinc oxide nanoparticles (Ni0.02Zn0.98O) photocatalyst supported on bentonite clay was investigated. ']


100%|██████████| 3/3 [00:00<00:00, 11.39it/s]


In [155]:
sorted_record = sorted(record.items(), key=lambda item:item[1]['sim'], reverse=True)
for_comparison = [i+1 for i in range(len(df['name']))]
comparison = []
for n in df['name']:
    for i, x in enumerate(sorted_record):
        if n == x[0]:
            comparison.append(i+1)
            record[n]['sim_rank'] = i+1
print(sorted_record[:5])

[('Bi2Te3', {'sim': 0.47818097472190857, 'token_num': 1, 'thermo_rank': 16, 'sim_rank': 1}), ('Sb2Te3', {'sim': 0.43377605080604553, 'token_num': 2, 'thermo_rank': 43, 'sim_rank': 2}), ('Cu2Se', {'sim': 0.3476353883743286, 'token_num': 2, 'thermo_rank': 1, 'sim_rank': 3}), ('Cu1.98Se', {'sim': 0.2988775372505188, 'token_num': 4, 'thermo_rank': 12, 'sim_rank': 4}), ('CeFe4Sb12', {'sim': 0.28699296712875366, 'token_num': 5, 'thermo_rank': 27, 'sim_rank': 5})]


In [156]:
print(for_comparison)
print(comparison)
res = stats.spearmanr(for_comparison, comparison)
print(res)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79]
[3, 30, 31, 72, 51, 19, 23, 14, 35, 22, 24, 4, 12, 79, 6, 1, 17, 65, 60, 21, 42, 26, 7, 41, 10, 13, 5, 20, 29, 62, 11, 61, 15, 50, 28, 34, 57, 45, 69, 67, 43, 56, 2, 48, 66, 16, 38, 59, 27, 46, 25, 75, 44, 63, 39, 74, 77, 49, 52, 76, 40, 32, 55, 47, 58, 68, 73, 18, 9, 64, 8, 53, 37, 70, 54, 78, 33, 36, 71]
SpearmanrResult(correlation=0.3961538461538462, pvalue=0.0003014824515726519)


In [157]:
sim_rank = []
token_num = []
for i in record.keys():
    sim_rank.append(record[i]['sim_rank'])
    token_num.append(record[i]['token_num'])
print(sim_rank)
print(token_num)
res = stats.spearmanr(sim_rank, token_num)
print(res)

[3, 30, 31, 72, 51, 19, 23, 14, 35, 22, 24, 4, 12, 79, 6, 1, 17, 65, 60, 21, 42, 26, 7, 41, 10, 13, 5, 20, 29, 62, 11, 61, 15, 50, 28, 34, 57, 45, 69, 67, 43, 56, 2, 48, 66, 16, 38, 59, 27, 46, 25, 75, 44, 63, 39, 74, 77, 49, 52, 76, 40, 32, 55, 47, 58, 68, 73, 18, 9, 64, 8, 53, 37, 70, 54, 78, 33, 36, 71]
[2, 7, 13, 6, 2, 9, 7, 8, 4, 6, 13, 4, 5, 3, 5, 1, 3, 4, 10, 7, 9, 9, 5, 8, 5, 5, 5, 10, 9, 2, 3, 6, 3, 13, 9, 9, 10, 10, 7, 7, 10, 7, 2, 9, 9, 10, 9, 10, 3, 7, 10, 4, 6, 10, 3, 10, 7, 1, 9, 5, 5, 7, 10, 6, 10, 10, 3, 5, 4, 10, 5, 4, 8, 3, 8, 1, 2, 7, 7]
SpearmanrResult(correlation=0.1701671509434133, pvalue=0.13378814764509875)


In [177]:
for_comparison = []
comparison = []
for i in record.keys():
    if record[i]['token_num'] in [7, 8, 9,10]:
        for_comparison.append(record[i]['thermo_rank'])
        comparison.append(record[i]['sim_rank'])
print(len(comparison))
res = stats.spearmanr(for_comparison, comparison)
print(res)

39
SpearmanrResult(correlation=0.5133603238866397, pvalue=0.0008311967403019619)


In [ ]:
import json

json_str = json.dumps(record, indent=4)
with open('zt_84_bert.json', 'w', encoding='utf-8') as json_file:
    json_file.write(json_str)